In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Iso Year'] = Query['datetime'].dt.isocalendar().year
    Src['Iso Week'] = Query['datetime'].dt.isocalendar().week
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Iso Year','Iso Week'], sort=0, group_keys=0)

    Calc['Week High']       = Calc_by['High'].max() .reset_index(drop=1)
    Calc['Week Low']        = Calc_by['Low'].min()  .reset_index(drop=1)
    Calc['HL > Avg']        = Calc['HL'] > Calc['HL'].mean()
    Calc['Is High of Wk']   =                Calc['Week High'] <= Calc['High']
    Calc['Is Low of Wk']    = Calc['Low'] <= Calc['Week Low']
    Calc['Is HoL of Wk']    = Calc['Is Low of Wk'] | Calc['Is High of Wk']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
Df = Calc[(Calc['Week.Day']==2) & (Calc['HL > Avg'])]   .reset_index(drop=1)

Df['Is HoL of Wk'].mean() * 100

np.float64(34.44444444444444)